In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
df.dropna(inplace=True)
df.columns = ['status', 'stay-time', 'Aspect-ratio', 'Density', 'Dis Variance', 'Velocity Mean', 'Velocity Variance',
              'a-Variance', 'maxDis', 'head-Variance', 'Class label']
df.head(5)

,status,stay-time,Aspect-ratio,Density,Dis Variance,Velocity Mean,Velocity Variance,a-Variance,maxDis,head-Variance,Class label
0,1.0,5.520556,8.594854,0.024205,3.278965,0.014234,0.008460,0.000056,167.689320,32.164128,1
1,1.0,3.850833,3.030676,0.051397,5.937132,0.034289,0.027529,0.000134,66.725869,5.636387,1
2,1.0,3.850556,6.788039,0.041447,3.433945,0.023077,0.014594,0.000103,110.791868,7.544310,1
3,1.0,4.348889,6.200071,0.011759,7.810434,0.029176,0.024673,0.000097,192.090427,9.990709,1
4,1.0,2.199444,2.410629,0.044884,5.714460,0.033267,0.025093,0.000141,45.923765,4.390358,1


In [9]:
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
X = df.iloc[:, 0:10].values
y = df.iloc[:, 10].values
seed = 0
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
model = RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100).fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy)


print('***********')
scores = cross_val_score(model, X_train, y_train, cv=10)
print(scores)  # 打印输出每次迭代的度量值（准确度）
print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2))  # 获取置信区间。（也就是均值和方差）
print()
scores = cross_val_score(model, X_train, y_train, cv=10)  
print('准确率：',np.mean(scores), scores)  
precisions = cross_val_score(model, X_train, y_train, cv=10, scoring='precision')  
print('精确率：', np.mean(precisions), precisions)  
recalls = cross_val_score(model, X_train, y_train, cv=10, scoring='recall')  
print('召回率：', np.mean(recalls), recalls)  
f1s = cross_val_score(model, X_train, y_train, cv=10, scoring='f1')  
print('综合评价指标：', np.mean(f1s), f1s)  

#max_depth=5, min_samples_leaf=19, min_samples_split=2, n_estimators=55, random_state=1
#Best: -0.095078 using {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000}

0.9638146039068695
***********
[0.96232242 0.96510191 0.96478221 0.96292864 0.96478221 0.97003398
 0.95644115 0.97034291 0.96477132 0.95982695]
Accuracy: 0.964 (+/- 0.008)

准确率： 0.9649366111319131 [0.96448425 0.96510191 0.96570899 0.96447328 0.96354649 0.97096077
 0.95613222 0.97126969 0.96755253 0.96013597]
精确率： 0.968092804798309 [0.96528926 0.97111111 0.97174515 0.9673853  0.96892342 0.97240618
 0.95630803 0.97666667 0.97065338 0.96043956]
召回率： 0.9681357328297603 [0.96742131 0.96576477 0.96742131 0.96466041 0.96576477 0.97404749
 0.96686913 0.97128658 0.96961326 0.96850829]
综合评价指标： 0.9680378176119339 [0.96886195 0.96842105 0.96983117 0.96731302 0.96570796 0.97353914
 0.96123178 0.97259895 0.96956281 0.96331034]


In [7]:
from sklearn.model_selection import train_test_split,cross_val_score,cross_validate
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
X = df.iloc[:, 0:10].values
y = df.iloc[:, 10].values
seed = 0
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
model = RandomForestClassifier(criterion='entropy', max_depth=50, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100).fit(X_train, y_train)
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print(accuracy)
feat_labels = df.columns[0:10]
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))


0.9631658617458373
 1) head-Variance                  0.522518
 2) maxDis                         0.116504
 3) status                         0.100584
 4) Density                        0.076785
 5) Dis Variance                   0.051192
 6) Velocity Mean                  0.031362
 7) Aspect-ratio                   0.029873
 8) a-Variance                     0.025583
 9) Velocity Variance              0.024101
10) stay-time                      0.021497


In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
X = df.iloc[:,0:10]
Y = df.iloc[:,10]
model = RandomForestClassifier(random_state=1)
param_grid = dict(max_depth=[5], min_samples_leaf=[19], min_samples_split=[2], n_estimators=[55])
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) #n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
grid_result = grid_search.fit(X_train, y_train)
pred = grid_result.predict(X_test)
print("最优测试集验证得分:"+str(np.mean(pred == y_test)))
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
#############################################################################################################
#Best: -0.187290 using {'max_depth': 5, 'min_samples_leaf': 19, 'min_samples_split': 2, 'n_estimators': 50}
#Best: -0.095078 using {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000}

最优测试集验证得分:0.9455056584732935


In [14]:
# Tune learning_rate
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
# load data
df = pd.read_csv(r'D:\航海\船舶停留轨迹分类模块\TrainingData\trainingData', header=None, engine="python")
# split data into X and y
X = df.iloc[:,0:10]
Y = df.iloc[:,10]
# grid search
model = RandomForestClassifier(random_state=1)
#param_grid = dict(n_estimators=[55], min_samples_leaf=np.arange(1,50,2), max_depth=np.arange(2,6), min_samples_split=np.arange(2,5))
#param_grid = dict(n_estimators=np.arange(20,100,5), min_samples_leaf=[19], max_depth=[5], min_samples_split=[2])
param_grid = [{'n_estimators':[10,100,1000],
'criterion':['entropy','gini'],
'max_depth':[10,50,100,200],
'min_samples_split':[2,5,10],
'min_weight_fraction_leaf':[0.0,0.1,0.2,0.3,0.4,0.5]
}]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold) #n_jobs: 并行数，int：个数,-1：跟CPU核数一致, 1:默认值
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']
for mean, param in zip(means, params):
    print("%f  with: %r" % (mean, param))
    
#Best: -0.187290 using {'max_depth': 5, 'min_samples_leaf': 19, 'min_samples_split': 2, 'n_estimators': 50}
#Best: -0.095078 using {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000}

Best: -0.095078 using {'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000}
-0.119909  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 10}
-0.112805  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100}
-0.112567  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000}
-0.299098  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 10}
-0.296273  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 100}
-0.303278  with: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.1, 'n_estimators': 1000}
-0.394881  with: {'criterion': 'ent